In [16]:
import os
from mistralai import Mistral
from pathlib import Path
import os
from mistralai import DocumentURLChunk, ImageURLChunk, TextChunk
import json

In [17]:
api_key = "METTEZ VOTRE CLE API ICI"
client = Mistral(api_key=api_key)

Exemple de OCR sur un fichier-test

In [14]:
pdf_file = Path("/Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/SciencePo_project/LG20/974/974-02/LG20-974-974-02-Rémy_MASSAIN-1-tour1-profession_foi.pdf")

In [23]:
uploaded_file = client.files.upload(
    file={
        "file_name": pdf_file.name,
        "content": pdf_file.read_bytes(),
    },
    purpose="ocr",
)

signed_url = client.files.get_signed_url(
    file_id=uploaded_file.id,
    expiry=1,
)

pdf_response = client.ocr.process(
    document={
        "type": "document_url",
        "document_url": signed_url.url,
    },
    model="mistral-ocr-latest",
)

In [24]:
pdf_response

OCRResponse(pages=[OCRPageObject(index=0, markdown='ELECTIONS LEGISLATIVES\n2ème circonscription\n20 et 27 septembre 2020\n\n# RÉMY MASSAIN\n\n![img-0.jpeg](img-0.jpeg)\n\n# DÉPUTÉ', images=[OCRImageObject(id='img-0.jpeg', top_left_x=569, top_left_y=675, bottom_right_x=1262, bottom_right_y=1952, image_base64=None, image_annotation=None)], dimensions=OCRPageDimensions(dpi=200, height=2522, width=1837), tables=[], hyperlinks=[], header=None, footer=None), OCRPageObject(index=1, markdown="- 2ème circonscription\n\n# ÉLECTIONS LÉGISLATIVES\n\nLe 20 et 27 septembre 2020\n\n# Rémy MASSAIN\n\nChères électrices, chers électeurs,\n\nDans la continuité de mes actions politiques, je m'engage à défendre les intérêts matériels et moraux de la population réunionnaise en élaborant des lois pour :\n\n1. le droit à l'emploi pour les jeunes et personnes handicapées ;\n2. la revalorisation des pensions des retraités, des bas salaires et des minimas sociaux ;\n3. le droit au logement et à la santé pour to

In [27]:
page0 = pdf_response.pages[0]
print(page0.model_dump())


{'index': 0, 'markdown': 'ELECTIONS LEGISLATIVES\n2ème circonscription\n20 et 27 septembre 2020\n\n# RÉMY MASSAIN\n\n![img-0.jpeg](img-0.jpeg)\n\n# DÉPUTÉ', 'images': [{'id': 'img-0.jpeg', 'top_left_x': 569, 'top_left_y': 675, 'bottom_right_x': 1262, 'bottom_right_y': 1952, 'image_base64': None, 'image_annotation': None}], 'dimensions': {'dpi': 200, 'height': 2522, 'width': 1837}, 'tables': [], 'hyperlinks': [], 'header': None, 'footer': None}


Le resultat textuel est compris dans "markdown" : 

In [29]:
for i, page in enumerate(pdf_response.pages, start=1):
    print(f"\n--- PAGE {i} ---")
    print(page.markdown)



--- PAGE 1 ---
ELECTIONS LEGISLATIVES
2ème circonscription
20 et 27 septembre 2020

# RÉMY MASSAIN

![img-0.jpeg](img-0.jpeg)

# DÉPUTÉ

--- PAGE 2 ---
- 2ème circonscription

# ÉLECTIONS LÉGISLATIVES

Le 20 et 27 septembre 2020

# Rémy MASSAIN

Chères électrices, chers électeurs,

Dans la continuité de mes actions politiques, je m'engage à défendre les intérêts matériels et moraux de la population réunionnaise en élaborant des lois pour :

1. le droit à l'emploi pour les jeunes et personnes handicapées ;
2. la revalorisation des pensions des retraités, des bas salaires et des minimas sociaux ;
3. le droit au logement et à la santé pour tous (revaloriser la CMU) ;
4. l'Aménagement durable du territoire, la protection de l'environnement et le développement des énergies renouvelables dans le cadre de la transition écologique ;
5. le soutien aux Petites et Moyennes Entreprises (PME) et aux Toutes Petites Entreprises (TPE) en allégeant notamment les charges sociales.

# Anaëlle HOAREAU

R

Lancer le code pour l'ensemble de pdf. 
Pour cela les fichiers doivent etre mis dans un seul dossier 

Ce code crée un tableau dont la première colonne indique le nom du fichier pdf, tandis que les autres colonnes contiennent le texte extrait des pages de chaque fichier.

In [20]:
import pandas as pd


In [6]:
PDF_DIR = Path("/Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/missing2000")  
# dossier avec des PDF

CSV_PATH = Path("/Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/OCR_missed.csv") 
#Chemin vers le CSV d'output : la place où on veut mettre le résultat d'OCR

In [14]:
if CSV_PATH.exists():
    df = pd.read_csv(CSV_PATH, encoding="utf-8")
    processed_files = set(df["filename"].astype(str))
    print(f"CSV trouvé: {len(processed_files)} fichiers traités")
else:
    df = pd.DataFrame()
    processed_files = set()
    print("CSV non trouvé, commence dès le début")


CSV trouvé: 1650 fichiers traités


In [8]:
def ocr_pdf_to_row(pdf_path: Path):
    uploaded_file = client.files.upload(
        file={
            "file_name": pdf_path.name,
            "content": pdf_path.read_bytes(),
        },
        purpose="ocr",
    )

    signed_url = client.files.get_signed_url(
        file_id=uploaded_file.id,
        expiry=1,
    )

    pdf_response = client.ocr.process(
        document={
            "type": "document_url",
            "document_url": signed_url.url,
        },
        model="mistral-ocr-latest",
    )

    row = {"filename": pdf_path.name}
    for i, page in enumerate(pdf_response.pages, start=1):
        row[f"page_{i}"] = page.markdown or ""
    return row


In [9]:
SAVE_EVERY = 100 
#Cet indicateur peut être modifié. Il oblige le code à enregistrer le résultat obtenu après chaque série de 100 fichiers.

from tqdm import tqdm

In [15]:
pdf_files = sorted(PDF_DIR.glob("*.pdf"))

new_rows = []
new_count = 0

pbar = tqdm(pdf_files, desc="OCR PDFs", unit="pdf", ascii=True)

for pdf_path in pbar:
    if pdf_path.name in processed_files:
        continue

    try:
        row = ocr_pdf_to_row(pdf_path)
        new_rows.append(row)
        processed_files.add(pdf_path.name)
        new_count += 1

        pbar.set_postfix(done=len(processed_files), batch=new_count)

        if new_count >= SAVE_EVERY:
            df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
            df.to_csv(CSV_PATH, index=False, encoding="utf-8")
            new_rows = []
            new_count = 0
            pbar.write(f"Saved -> {CSV_PATH}")

    except Exception as e:
        pbar.write(f"FAIL: {pdf_path.name} -> {e}")

# сохранить остаток
if new_rows:
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8")
    pbar.write(f"Final save -> {CSV_PATH}")

df


OCR PDFs:  50%|#############################################################8                                                              | 1044/2093 [00:11<00:15, 69.51pdf/s, batch=1, done=1651]

FAIL: LG22-93-05-tour1-10-Amel HORMI-profession_foi_falc.pdf -> API error occurred: Status 500. Body: {"object":"error","message":"Service unavailable.","type":"internal_server_error","param":null,"code":"1000"}


OCR PDFs:  84%|######################################################################################################                    | 1751/2093 [05:35<17:25,  3.06s/pdf, batch=100, done=1750]

Saved -> /Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/OCR_missed.csv


OCR PDFs:  88%|###########################################################################################################8              | 1851/2093 [10:58<13:01,  3.23s/pdf, batch=100, done=1850]

...

OCR PDFs:  98%|#######################################################################################################################5  | 2051/2093 [21:30<02:19,  3.33s/pdf, batch=100, done=2050]

Saved -> /Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/OCR_missed.csv


OCR PDFs: 100%|###########################################################################################################################| 2093/2093 [23:50<00:00,  1.46pdf/s, batch=42, done=2092]

Final save -> /Users/quentinnippert/Documents/hackaton_week/professions-foi-candidats/OCR_missed.csv


,filename,page_1,page_2,page_3,page_4,page_5,page_6,page_7,page_8,page_9,page_10,page_11,page_12,page_13,page_14
0,LG20-974-974-02-Hary_GRONDIN-10-tour1-professi...,LEGISLATIVE PARTIELLE - SEPTEMBRE 2020\n\n# HU...,Programme de l'an I Covid du candidat\nHARY GR...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LG20-974-974-02-Jean_François_NATIVEL-12-tour1...,# Législatives partielles des 20 et 27 Septemb...,"# POUR UN VRAI CHANGEMENT, VOTEZ JEAN FRANÇOIS...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LG20-974-974-02-Rémy_MASSAIN-1-tour1-professio...,ELECTIONS LEGISLATIVES\n2ème circonscription\n...,- 2ème circonscription\n\n# ÉLECTIONS LÉGISLAT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LG22-01-01-tour1-1-Vincent GUILLERMIN-professi...,ÉLECTIONS LÉGISLATIVES - 12 ET 19 JUIN 2022\n\...,Nos priorités pour le territoire\n\nDes rives ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LG22-01-01-tour1-1-Vincent GUILLERMIN-professi...,ÉLECTIONS LÉGISLATIVES - 12 ET 19 JUIN 2022\n\...,Nos priorités pour le territoire\n\nDes rives ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,LG24-ZZ-ZZ08-tour2-6-Caroline YADAN_profession...,ÉLECTIONS LÉGISLATIVES DES 30 JUIN ET 7 JUILLE...,# MES PRIORITÉS POUR VOUS\n\n## SIMPLIFICATION...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2088,LG24-ZZ-ZZ09-tour1-6-Rachid TAHIRI_profession_...,"# UNION DE LA GAUCHE\n\nPOPULAIRE, PROGRESSIST...","À vos côtés,\n\nJE M'ENGAGE\n\n# Pour les Fran...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2089,LG24-ZZ-ZZ10-tour2-6-Elsa DI MEO_profession_fo...,NOUVEAU FRONT POPULAIRE\n\nCandidate : Elsa di...,# QUI SOMMES-NOUS ?\n\nLe **Nouveau Front Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2090,LG24-ZZ-ZZ11-tour2-13-Anne GENETET_profession_...,ÉLECTIONS LÉGISLATIVES - 2d TOUR DIMANCHE 7 JU...,"notamment l'obsession d'un ""impôt universel"" q...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
